In [ ]:
import chess
from chessboard import display
import time

# Represent game state (board + which player's turn)
class State:
    def __init__(self, board=None, white_to_move=True):
        self.board = chess.Board() if board is None else board
        self.white_to_move = white_to_move

    def is_over(self):
        return self.board.is_game_over()

    def next_states(self):
        states = []
        for mv in self.board.legal_moves:
            new_b = self.board.copy()
            new_b.push(mv)
            states.append(State(new_b, not self.white_to_move))
        return states

    def evaluate(self):
        # terminal cases first
        if self.board.is_checkmate():
            return -10000 if self.white_to_move else 10000
        if self.board.is_stalemate() or self.board.is_insufficient_material() or self.board.can_claim_draw():
            return 0

        values = {chess.PAWN: 1, chess.KNIGHT: 3, chess.BISHOP: 3,
                  chess.ROOK: 5, chess.QUEEN: 9, chess.KING: 0}
        score = 0

        # material
        for _, piece in self.board.piece_map().items():
            val = values[piece.piece_type]
            score += val if piece.color == chess.WHITE else -val

        # central control
        for sq in [chess.D4, chess.E4, chess.D5, chess.E5]:
            p = self.board.piece_at(sq)
            if p:
                score += 0.25 if p.color == chess.WHITE else -0.25

        # mobility
        tmp = self.board.copy()
        tmp.turn = chess.WHITE
        w_moves = len(list(tmp.legal_moves))
        tmp.turn = chess.BLACK
        b_moves = len(list(tmp.legal_moves))
        score += 0.05 * (w_moves - b_moves)

        # king safety
        wk = self.board.king(chess.WHITE)
        bk = self.board.king(chess.BLACK)
        if wk:
            score -= 0.5 * len(self.board.attackers(chess.BLACK, wk))
        if bk:
            score += 0.5 * len(self.board.attackers(chess.WHITE, bk))

        # bishop pair bonus
        wb = sum(1 for p in self.board.piece_map().values() if p.piece_type == chess.BISHOP and p.color == chess.WHITE)
        bb = sum(1 for p in self.board.piece_map().values() if p.piece_type == chess.BISHOP and p.color == chess.BLACK)
        if wb >= 2: score += 0.25
        if bb >= 2: score -= 0.25

        return score


def minimax(state, depth, a, b, maximize, limit):
    if state.is_over() or depth == limit:
        return state.evaluate(), None

    best_mv = None
    if maximize:  # White
        val = float('-inf')
        for nxt in state.next_states():
            sc, _ = minimax(nxt, depth+1, a, b, False, limit)
            if sc > val:
                val, best_mv = sc, nxt.board.peek()
            a = max(a, sc)
            if a >= b: break
        return val, best_mv
    else:  # Black
        val = float('inf')
        for nxt in state.next_states():
            sc, _ = minimax(nxt, depth+1, a, b, True, limit)
            if sc < val:
                val, best_mv = sc, nxt.board.peek()
            b = min(b, sc)
            if a >= b: break
        return val, best_mv


def play():
    state = State()
    depth_limit = 3
    gui = display.start()

    print("You play White. Enter moves like 'e2e4'. Type 'quit' to stop.")

    while not state.is_over():
        display.update(state.board.fen(), gui)
        if display.check_for_quit():
            break

        if state.white_to_move:
            mv = input("Your move: ")
            if mv == "quit": break
            try:
                move = chess.Move.from_uci(mv)
                if move in state.board.legal_moves:
                    b = state.board.copy()
                    b.push(move)
                    state = State(b, False)
                else:
                    print("Not legal, try again.")
            except:
                print("Bad format, use UCI like e2e4.")
        else:
            print("AI is thinking...")
            t0 = time.time()
            val, best = minimax(state, 0, float('-inf'), float('inf'), False, depth_limit)
            t1 = time.time()
            print(f"AI took {t1-t0:.2f}s (eval {val:.2f})")
            if best:
                b = state.board.copy()
                b.push(best)
                state = State(b, True)
                print("AI plays:", best.uci())
            else:
                break

    print("Game over!")
    display.update(state.board.fen(), gui)
    if state.board.is_checkmate():
        print("Checkmate! Winner:", "White" if not state.white_to_move else "Black")
    elif state.board.is_stalemate(): print("Stalemate.")
    elif state.board.is_insufficient_material(): print("Draw: not enough material.")
    elif state.board.can_claim_draw(): print("Draw by rule.")
    time.sleep(2)
    display.terminate()


if __name__ == "__main__":
    play()


You play White. Enter moves like 'e2e4'. Type 'quit' to stop.
